In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import sys
sys.path.append("/content/drive/MyDrive/Colab Notebooks")


In [ ]:
cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import linalg as LA
import pandas as pd
#import pysindy as ps
from tqdm import tqdm
from termcolor import colored
from scipy.optimize import minimize
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
import yaml
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import train_test_split


from PolyDiff import PolyDiffPoint
from mpl_toolkits.mplot3d import Axes3D

%matplotlib inline
plt.rcParams["font.family"] = 'Arial'
np.set_printoptions(suppress=True)

In [ ]:
class FitEqu(object):
    '''
    Sparse regression to fit a equation

    1. create dataset
    2. preprocess dataset
    3. build library
    4. fit equation
    '''
    def __init__(self):
        super(FitEqu, self).__init__()

    @staticmethod
    def load_dataset(dataset_dir):
        '''
        load dataset
        '''
        data=pd.read_csv(dataset_dir)
        U = np.array(data['U:0']).reshape(100,63,39)
        V = np.array(data['U:1']).reshape(100,63,39)
        P = np.array(data['p']).reshape(100,63,39)

        A=np.zeros((100*63*39,1))
        for i in range(0,len(A)):
          A[i]=101325
        A=A.reshape(100,63,39)
        P = P+A
        P = np.transpose(P, axes=[1,2,0])
        U = np.transpose(U, axes=[1,2,0])
        V = np.transpose(V, axes=[1,2,0])

        return U, V, P




    @staticmethod
    def non_dimensionlize(U, V, P, dx, dy, dt, ref_params, sim_params):
        '''
        non-dimensionlize dx, dy, and dt
        '''
        # reference parameters
        l_ref, v_ref,p_ref= ref_params['l_ref'], ref_params['v_ref'],ref_params['p_ref']
        r_ref=sim_params['rho']
        t_ref, w_ref = l_ref / v_ref, v_ref / l_ref

        # non-dimensionlize data
        U, V, P = U/v_ref, V/v_ref, (P*r_ref)/p_ref
        dx, dy, dt = dx / l_ref, dy / l_ref, dt / t_ref

        sim_params['mu']=sim_params['niu']*sim_params['rho']
        # analyze the dimensionless numbers
        Re = v_ref * l_ref * sim_params['rho'] / sim_params['mu']
        Fr = ref_params['v_ref'] / float(np.sqrt(sim_params['g'] * ref_params['l_ref']))
        Eu = ref_params['p_ref'] / (sim_params['rho'] * ref_params['v_ref']**2)
        pi_group = {'Re': Re, 'Fr': Fr, 'Eu': Eu}
        print(colored(f'Re: {int(Re)}, coef_best: {round(1 / Re, 6)}', 'red'))
        print(colored(f'Eu: {int(Eu)}, coef_best: {round(Eu, 6)}', 'red'))
        # print(colored(f'Fr: {int(Fr)}, coef_best: {round(1/Fr/Fr, 6)}', 'red'))
        return U, V, P, dx, dy, dt, pi_group

    @staticmethod
    def sample_points(Nx, Ny, boundary_x, boundary_y, boundary_t):
        '''
        sample a certain number of points to train
        '''
        np.random.seed(0)
        points, count = {}, 0
        num_xy = 300
        for _ in range(num_xy):
            x = np.random.choice(np.arange(boundary_x, Nx-boundary_x), 1)[0]
            y = np.random.choice(np.arange(boundary_y, Ny-boundary_y), 1)[0]
            for t in range(10, 100, 10):
                points[count] = [x, y, t]
                count = count + 1
        return points

    @staticmethod
    def cal_derivatives(U, V, P, points, boundary_x, boundary_y, boundary_t, dx, dy, dt, deg=5):
        num_points = len(points)
        p = np.zeros((num_points,1))
        u = np.zeros((num_points,1))
        v = np.zeros((num_points,1))
        ut, vt = np.zeros((num_points,1)), np.zeros((num_points,1))
        ux, vx = np.zeros((num_points,1)), np.zeros((num_points,1))
        uy, vy = np.zeros((num_points,1)), np.zeros((num_points,1))
        uxx, vxx = np.zeros((num_points,1)), np.zeros((num_points,1))
        uxy, vxy = np.zeros((num_points,1)), np.zeros((num_points,1))
        uyy, vyy = np.zeros((num_points,1)), np.zeros((num_points,1))
        px, py = np.zeros((num_points,1)), np.zeros((num_points,1))

        Nx_sample, Ny_sample, Nt_sample = 2*boundary_x-1, 2*boundary_y-1, 2*boundary_t-1

        for idx_p in tqdm(points.keys()):
            [x, y, t] = points[idx_p]
            u[idx_p] = U[x, y, t]
            v[idx_p] = V[x, y, t]
            p[idx_p] = P[x, y, t]

            ut_part = U[x, y, t-int((Nt_sample-1)/2):t+int((Nt_sample+1)/2)]
            ux_part = U[x-int((Nx_sample-1)/2):x+int((Nx_sample+1)/2), y, t]
            uy_part = U[x, y-int((Ny_sample-1)/2):y+int((Ny_sample+1)/2), t]
            ux_part_yp = U[x-int((Nx_sample-1)/2):x+int((Nx_sample+1)/2), y+1, t]
            ux_part_ym = U[x-int((Nx_sample-1)/2):x+int((Nx_sample+1)/2), y-1, t]

            vt_part = V[x, y, t-int((Nt_sample-1)/2):t+int((Nt_sample+1)/2)]
            vx_part = V[x-int((Nx_sample-1)/2):x+int((Nx_sample+1)/2), y, t]
            vy_part = V[x, y-int((Ny_sample-1)/2):y+int((Ny_sample+1)/2), t]
            vx_part_yp = V[x-int((Nx_sample-1)/2):x+int((Nx_sample+1)/2), y+1, t]
            vx_part_ym = V[x-int((Nx_sample-1)/2):x+int((Nx_sample+1)/2), y-1, t]

            px_part = P[x-int((Nx_sample-1)/2):x+int((Nx_sample+1)/2), y, t]
            py_part = P[x, y-int((Ny_sample-1)/2):y+int((Ny_sample+1)/2), t]

            ut[idx_p] = PolyDiffPoint(ut_part, np.arange(Nt_sample)*dt, deg, 1)[0]
            ux_diff = PolyDiffPoint(ux_part, np.arange(Nx_sample)*dx, deg, 2)
            uy_diff = PolyDiffPoint(uy_part, np.arange(Ny_sample)*dy, deg, 2)
            ux_diff_yp = PolyDiffPoint(ux_part_yp, np.arange(Nx_sample)*dx, deg, 2)
            ux_diff_ym = PolyDiffPoint(ux_part_ym, np.arange(Nx_sample)*dx, deg, 2)

            vt[idx_p] = PolyDiffPoint(vt_part, np.arange(Nt_sample)*dt, deg, 1)[0]
            vx_diff = PolyDiffPoint(vx_part, np.arange(Nx_sample)*dx, deg, 2)
            vy_diff = PolyDiffPoint(vy_part, np.arange(Ny_sample)*dy, deg, 2)
            vx_diff_yp = PolyDiffPoint(vx_part_yp, np.arange(Nx_sample)*dx, deg, 2)
            vx_diff_ym = PolyDiffPoint(vx_part_ym, np.arange(Nx_sample)*dx, deg, 2)

            ux[idx_p], uxx[idx_p] = ux_diff[0], ux_diff[1]
            uy[idx_p], uyy[idx_p] = uy_diff[0], uy_diff[1]
            uxy[idx_p] = (ux_diff_yp[0] - ux_diff_ym[0]) / (2 * dy)

            vx[idx_p], vxx[idx_p] = vx_diff[0], vx_diff[1]
            vy[idx_p], vyy[idx_p] = vy_diff[0], vy_diff[1]
            vxy[idx_p] = (vx_diff_yp[0] - vx_diff_ym[0]) / (2 * dy)

            px[idx_p] = PolyDiffPoint(px_part, np.arange(Nx_sample)*dx, deg, 1)[0]
            py[idx_p] = PolyDiffPoint(py_part, np.arange(Ny_sample)*dy, deg, 1)[0]

        base_library = [u, v, p, ut, ux, uy, uxx, uxy, uyy, vt, vx, vy, vxx, vxy, vyy, px, py]
        return base_library

    @staticmethod
    def parse_library(base_library, sim_params, ref_params):
        '''
        calculate derivatives and build library for sparse regression
        '''
        u, v, p, ut, ux, uy, uxx, uxy, uyy, vt, vx, vy, vxx, vxy, vyy, px, py = base_library
        g_x = np.zeros_like(u)
        g_y = np.ones_like(u) * sim_params['g'] * ref_params['l_ref'] / ref_params['v_ref']**2
        # ########################standard sindy library#########################
        # X_library = [u*ux, v*uy, uxx, uyy, px]
        # names = ['u*ux', 'v*uy', 'uxx', 'uyy', 'px']

        # #########################rotationial invariance#########################
        ########################## More terms (6 terms in total, 4 independent terms)##########################
        ut_ = np.concatenate([ut+vt, vt+ut])
        u_ = np.concatenate([u+v, v+u])
        uu_ = np.concatenate([u*(u+v), v*(v+u)])
        vu_ = np.concatenate([v*(u+v), u*(v+u)])

        ux_ = np.concatenate([ux+vx, vy+uy])
        uy_ = np.concatenate([uy+vy, vx+ux])
        uxx_ = np.concatenate([uxx+vxx, vyy+uyy])
        uyy_ = np.concatenate([uyy+vyy, vxx+uxx])

        uux_ = np.concatenate([u*(ux+vx), v*(vy+uy)])
        uuy_ = np.concatenate([u*(uy+vy), v*(vx+ux)])
        uuxx_ = np.concatenate([u*(uxx+vxx), v*(vyy+vyy)])
        uuyy_ = np.concatenate([u*(uyy+vyy), v*(vxx+uxx)])

        vux_ = np.concatenate([v*(ux+vx), u*(vy+uy)])
        vuy_ = np.concatenate([v*(uy+vy), u*(vx+ux)])
        vuxx_ = np.concatenate([v*(uxx+vxx), u*(vyy+vyy)])
        vuyy_ = np.concatenate([v*(uyy+vyy), u*(vxx+uxx)])

        uuxx_ = np.concatenate([u*(uxx+vxx), v*(vyy+vyy)])
        uuyy_ = np.concatenate([u*(uyy+vyy), v*(vxx+uxx)])
        vuxx_ = np.concatenate([v*(uxx+vxx), u*(vyy+vyy)])
        vuyy_ = np.concatenate([v*(uyy+vyy), u*(vxx+uxx)])

        p_ = np.concatenate([p, p])
        px_ = np.concatenate([px+py, py+px])

        ut, u, uu, vu, ux, uy, uxx, uyy = ut_, u_, uu_, vu_, ux_, uy_, uxx_, uyy_
        uux, uuy, uuxx, uuyy, vux, vuy, vuxx, vuyy, px = uux_, uuy_, uuxx_, uuyy_, vux_, vuy_, vuxx_, vuyy_, px_
        p = p_
        mu = np.ones_like(u)*sim_params['niu']*sim_params['rho']
        diameter=np.ones_like(u)*ref_params['l_ref']
        v_init=np.ones_like(u)*ref_params['v_ref']
        density=np.ones_like(u)*sim_params['rho']
        p_init=np.ones_like(u)*ref_params['p_ref']


        # term and position: (uux: 7, vuy: 12, uxx: 5, uyy: 6, px: 16)
        X_library = [
            u, uu, vu, ux, uy,
            uxx, uyy, uux, uuy, uuxx,
            uuyy, vux, vuy, vuxx, vuyy,
            p, px]
        names = ['u', 'uu', 'vu', 'ux', 'uy', 'uxx', 'uyy', 'uux',
                'uuy', 'uuxx', 'uuyy', 'vux', 'vuy', 'vuxx', 'vuyy', 'p', 'px', 'mu', 'diameter', 'v_init', 'density', 'p_init']
        xx=[ mu, diameter, v_init, density, p_init]

        # X_library = [uux, vuy, uxx, uyy, px]
        # names = ['uux', 'vuy', 'uxx', 'uyy', 'px']
        ##########################Reshape data##########################
        X_library = np.squeeze(np.stack(X_library, axis=-1))
        xx = np.squeeze(np.stack(xx, axis=-1))
        y_library = ut.reshape(-1, 1)

        return X_library, y_library, names, xx

    @staticmethod
    def normalization(X_library, y_library):
        '''
        Rescale the data by each column
        rescale the data by the absolute mean for each column
        '''
        norm_coef = np.mean(np.abs(np.mean(X_library, axis=0)))
        X_library = X_library / norm_coef
        y_library = y_library / norm_coef
        return X_library, y_library, norm_coef

    @staticmethod
    def check_library(X_library, y_library):
        '''
        check whether the library has any Nan
        '''
        if np.any(np.isnan(X_library)) or np.any(np.isnan(y_library)):
            print('Nan exists in library.')
            return True
        return False

    @staticmethod
    def check_equ(X_library, y_library, pi_group):
        '''
        check the r2 for the target equation
        '''
        pred_best = - X_library[:, 7] - X_library[:, 12]\
                    + 1/pi_group['Re'] * (X_library[:, 5] + X_library[:, 6]) \
                    - X_library[:, 16] * pi_group['Eu']
        pred_best = pred_best.reshape(-1,1)
        r2 = r2_score(y_library, pred_best)
        print(f"Analytical r2_score: {round(r2, 6)}")

        pred_best = - X_library[:, 7] - X_library[:, 12]\
                    - X_library[:, 16] * pi_group['Eu']
        pred_best = pred_best.reshape(-1,1)
        r2_2 = r2_score(y_library, pred_best)
        print(f"Analytical r2_score (no 1/Re): {round(r2_2, 6)}")
        print('difference', r2-r2_2)
        return None

In [ ]:
config_path = '/content/drive/MyDrive/Colab Notebooks/config_Euler_2cylinder_clean.yml.'

config = yaml.load(open(config_path, 'r'), Loader=yaml.FullLoader)
case_id_list = list(config['case_id_list'].keys())



def prepare_dataset(is_show=False):
    '''
    prepare a sets of dataset
    '''
    res = []
    fit_equ = FitEqu()
    c=[]
    d=[]
    e=[]
    c=pd.DataFrame(c)
    d=pd.DataFrame(d)
    e=pd.DataFrame(e)


    for case_id in case_id_list:

        print('*' * 40, case_id)
        case_info = config['case_id_list'][case_id]
        Nx, Ny = 200, 100
        dx, dy = 6/Nx, 3/Ny

        ref_params = case_info['ref_params']
        sim_params = case_info['sim_params']

        sim_params['mu']=sim_params['niu']*sim_params['rho']
        dt = sim_params['dt']

        # parameters for sparse regression
        #x_range, y_range = case_info['x_range'], case_info['y_range']
        boundary_x, boundary_y, boundary_t = case_info['fitting']['boundary_num']  # default is 5
        deg = case_info['fitting']['deg']               # polynomial degree default is 3

        #####################Prepare Dataset#####################
        # generate data and save to a folder
        U, V, W = fit_equ.load_dataset(case_info['dataset_dir'])
        #U, V, W = fit_equ.select_sub_domain(U, V, W, x_range, y_range)
        # non-dimensionlize data and dx, dy, dt
        U, V, W, dx, dy, dt, pi_group = fit_equ.non_dimensionlize(U, V, W, dx, dy, dt, ref_params, sim_params)
        Nx, Ny, Nt = W.shape
        # sample points
        points = fit_equ.sample_points(Nx, Ny, boundary_x, boundary_y, boundary_t)
        # plot the 1st frame
        # if is_show: fig = plt.figure(figsize=(6, 4)); plt.imshow(V[:, :, 0].T)

        #####################Prepare library#####################
        base_library = fit_equ.cal_derivatives(
            U, V, W, points, boundary_x, boundary_y, boundary_t, dx, dy, dt, deg)

        X_library, y_library, names, xx = fit_equ.parse_library(base_library, sim_params, ref_params)
        X_library, y_library, norm_coef = fit_equ.normalization(X_library, y_library)

        X_library=pd.DataFrame(X_library)
        c=pd.concat([c,X_library],ignore_index=True)
        xx=pd.DataFrame(xx)
        e=pd.concat([e,xx],ignore_index=True)
        y_library=pd.DataFrame(y_library)
        d=pd.concat([d,y_library],ignore_index=True)

    X_library=c.values
    y_library=d.values
    xx=e.values
    return X_library, y_library, xx
X_library, y_library, xx = prepare_dataset(True)



**************************************** v2-Re-2000
Re: 2000, coef_best: 0.0005
Eu: 16000, coef_best: 16000.0


100%|██████████| 2700/2700 [00:19<00:00, 139.28it/s]


**************************************** v1-Re-2600
Re: 2600, coef_best: 0.000385
Eu: 10355, coef_best: 10355.029586


100%|██████████| 2700/2700 [00:16<00:00, 165.69it/s]


**************************************** v3-Re-3000
Re: 3000, coef_best: 0.000333
Eu: 5341, coef_best: 5341.880342


100%|██████████| 2700/2700 [00:22<00:00, 120.32it/s]


In [ ]:
print(X_library.shape,y_library.shape,xx.shape)
X1_library=X_library[0:5400]
X2_library=X_library[5400:10800]
X3_library=X_library[10800:16200]
y1_library=y_library[0:5400]
y2_library=y_library[5400:10800]
y3_library=y_library[10800:16200]
X1=np.concatenate((X1_library,y1_library),axis=1)
X2=np.concatenate((X2_library,y2_library),axis=1)
X3=np.concatenate((X3_library,y3_library),axis=1)
X1=pd.DataFrame(X1)
X2=pd.DataFrame(X2)
X3=pd.DataFrame(X3)
X1.to_csv('/content/drive/MyDrive/Colab Notebooks/EulerX1.csv')
X2.to_csv('/content/drive/MyDrive/Colab Notebooks/EulerX2.csv')
X3.to_csv('/content/drive/MyDrive/Colab Notebooks/EulerX3.csv')

aaa=pd.DataFrame(X_library)
aaa.to_csv('/content/drive/MyDrive/Colab Notebooks/Euler自变量.csv')
bbb=pd.DataFrame(y_library)
bbb.to_csv('/content/drive/MyDrive/Colab Notebooks/Euler因变量.csv')
ccc=pd.DataFrame(xx)
ccc.to_csv('/content/drive/MyDrive/Colab Notebooks/Euler量纲数.csv')


(16200, 17) (16200, 1) (16200, 5)
